# Projeto - 2 
## Juliana Pessoa, Vinicius Bonelli, Vitória Rocha

### Introdução

O trabalho realizado, conhecido como teste A/B, teve como proposta analisar o tempo que os usuários levavam para encontrar determinado tópico em um site de nossa escolha. Para isso, o site foi adaptado para dois modelos: tipo A e tipo B. Em cada um, o tópico a ser encontrado localizava-se em lugares diferentes.

O website escolhido foi o portfolio criado no Projeto 1 do curso de Co-Design de Aplicativos, disponível nesse link https://categoriacd.firebaseapp.com/.

### Projeto

Foram criadas duas versões do site, ambas com a primeira página igual. As páginas da aba "Projeto" são diferentes no método de categorização dos projetos. As páginas seguintes foram modificadas muito pouco, e de maneira geral, são muito próximas.

Logo na página de abertura, é sorteada, aleatoriamente, uma das duas versões da página, A e B, como mencionado acima, e o usuário é instruído a encontrar no site o projeto "Estação Meteorológica". 

Na versão A, o projeto encontra-se na categoria "Arduíno", enquanto na versão B, encontra-se na categoria "Eletrônico".

Em seguida, são medidos os tempos que o usuário leva para achar a categoria certa do projeto, o tempo para achar o projeto depois de achar a categoria e o tempo total desde o início até achar o projeto.

#### Porque escolhemos a mudança de categoria?

Escolhemos a mudança de categoria devido a amplitude que consistiu o projeto Estação Meteorológica, uma vez que este pode ser inserido tanto na categoria "Arduíno" quanto na categoria "Eletrônico" e, dessa forma, será avaliado como o usuário identifica o projeto dentro das opções de categorias apresentadas.

### A análise estatística

A análise será feita com base no Teorema do Limite Central (TLC), sendo a exata ferramenta discutida mais a frente, dependendo do número de amostras que obtivermos.

Como o número de amostras era pequeno e não tinhamos informações sobre o desvio padrão populacional, não podíamos assumir que sua distribuição era dada por uma normal e utilizamos a distribuição T-student para corrigir distorções causadas pela falta do desvio padrão populacional.

In [58]:
import pandas as pd
import json
import itertools
import matplotlib.pyplot as plt
import scipy.stats
%matplotlib inline

# Manipulando JSON

In [64]:
arquivo = open("categoriacd-export.json")
linhas = arquivo.readlines()
texto = "".join(linhas)
dados_fire = json.loads(texto)

In [65]:
lista_dicionarios = []

for k,v in dados_fire["logs"].items():
    lista_dicionarios.append(v)

dataframe = pd.DataFrame.from_records(lista_dicionarios)    

dataframe

,tempo,tipo,user_id,versao
0,1479351735583,inicio,3137998881,A
1,1479345303606,final,3723105756,B
2,1478481355889,final,455822798,A
3,1478285321803,inicio,818354266,A
4,1479369743204,final,3936828694,A
5,1479345591168,inicio,978031372,B
6,1479347115072,final,3796913893,A
7,1479345504799,inicio,2419417467,A
8,1479378854855,inicio,4261560382,B
9,1478357257365,inicio,3729058087,A


In [66]:
agrupado = dataframe.groupby(by=["user_id", "versao"])
users = set(dataframe.user_id)
users = list(users) 
resultados = []

for u in users:
    for versao in ["A", "B"]:
        if (u, versao) in agrupado.groups.keys(): # Check if a specific pair user/type of test happened
            grupo1 =  agrupado.get_group((u, versao))
            grupo1 = grupo1.sort_values(by="tempo") # Para evitar que inícios sem fim gerem tempos negativos
            
            if len(grupo1[grupo1.tipo=="final"])>= 1 and len(grupo1[grupo1.tipo=="inicio"]) >= 1:
                delta_t = grupo1[grupo1.tipo=="final"].iloc[-1].tempo - grupo1[grupo1.tipo=="inicio"].iloc[0].tempo
                resultados.append({"user_id":u, "delta_t":delta_t, "versao":versao})
                
dataframe_resumo = pd.DataFrame.from_records(resultados)

dataframe_resumo.delta_t/=1000 # Convertendo milissegundos em segundos

dataframe_resumo

,delta_t,user_id,versao
0,20.348,3857193483,B
1,20.368,1686436492,B
2,127.473,2766741010,A
3,37.600,3936828694,A
4,28.412,542355478,B
5,50.978,2244328473,B
6,26.169,2991394204,A
7,973.810,3774183163,A
8,138.359,3774183163,B
9,16.863,1431168288,A


# Análise estatística

## Procedimentos

Para analisar estatisticamente, optou-se por realizar testes de hipótese a fim de tomar uma conclusão. Primeiramente,  calculou-se a média amostral (µ), o desvio padrão (σ) e a variância (σ²). Encontrada a média amostral µ = 44.537, estabeleceu-se os testes de hipótese para tomada de decisão: a hipótese nula (H0) como H0 <= 44.537, e a hipótese alternativa (H1) como H1 > 44.537.

A segunda etapa foi a realização da Distribuição T, semelhante à Normal, mas com caudas mais espessas. O nível de significância adotado foi o de 5%. Analisamos as amostras de A e de B para vermos se alguma versão do site refutava nossa hipótese.

Para realizarmos todos os testes tivemos que manipular os dados obtidos. Como o site era bem básico e as funções de armazenar tempo bem simples optamos por filtrar posteriormente alguns dados como por exemplo deltas de tempo muito grandes ou negativos. Assumimos que se a pessoa demorasse mais de 5 minutos para responder o teste ela estava inativa e a partir dai filtramos.

In [62]:
tempos_A = list(dataframe_resumo[dataframe_resumo.versao=="A"].delta_t)
tempos_B = list(dataframe_resumo[dataframe_resumo.versao=="B"].delta_t)

In [68]:
#filtrando resultados aberrantes
for i in tempos_A:
    if i < 0:
        tempos_A.remove(i)
    if i > 300:
        tempos_A.remove(i)
        
for i in tempos_B:
    if i < 0:
        tempos_B.remove(i)
    if i > 300:
        tempos_B.remove(i)
        

# Médias, desvios e variâncias

In [69]:
#calculando media, desvio padrao e variancia de A
somaA = 0
for i in tempos_A:
    somaA += i
mediaA = somaA / len(tempos_A)

somaA = 0
for i in tempos_A:
    somaA += (i - mediaA) ** 2
desvioA = (somaA / len(tempos_A)) ** (1 / 2)

varA = desvioA ** 2

print("A media amostral de A é :{0}".format(mediaA))
print("O desvio amostral de A é :{0}".format(desvioA))
print("A variância amostral de A é :{0}".format(varA))

A media amostral de A é :46.27719047619047
O desvio amostral de A é :61.56936077351265
A variância amostral de A é :3790.7861860589583


In [70]:
#calculando media, desvio padrao e variancia de B
somaB = 0
for i in tempos_B:
    somaB += i
mediaB = somaB / len(tempos_B)

somaB = 0
for i in tempos_B:
    somaB += (i - mediaB) ** 2
desvioB = (somaB / len(tempos_B)) ** (1 / 2)

varB = desvioB ** 2

print("A media amostral de B é :{0}".format(mediaB))
print("O desvio amostral de B é :{0}".format(desvioB))
print("A variância amostral de B é :{0}".format(varB))

A media amostral de B é :42.50672222222222
O desvio amostral de B é :31.939199003393856
A variância amostral de B é :1020.1124329783951


In [71]:
tempos_AB = []
for i in tempos_A:
    tempos_AB.append(i)
for i in tempos_B:
    tempos_AB.append(i)

#calculando media, desvio padrao e variancia de amostra total
somaAB = 0
for i in tempos_AB:
    somaAB += i
mediaAB = somaAB / len(tempos_AB)

somaAB = 0
for i in tempos_AB:
    somaAB += (i - mediaAB) ** 2
desvioAB = (somaAB / len(tempos_AB)) ** (1 / 2)

varAB = desvioAB ** 2

print("A media amostral é :{0}".format(mediaAB))
print("O desvio amostral é :{0}".format(desvioAB))
print("A variância amostral é :{0}".format(varAB))

A media amostral é :44.536974358974334
O desvio amostral é :50.15522666809061
A variância amostral é :2515.546762127548


# Definindo as hipóteses

H0 : u <= 44.537
Ha : u > 44.537 
95% de confiança


# t-student e valor-p

In [72]:
x = mediaB
u = mediaAB
s = desvioB
n = len(tempos_B)
#calculando graus de liberdade
v = n - 1
print("O número de graus de liberdade é de : {0}".format(v))
#testando para ver se versao B rejeita a hipotese
t = (x - u) / (s / (n ** (1 / 2)))
print("O valor de t observado é de : {0}".format(t))
#calculando o valor-p
valorp = scipy.stats.t.cdf(t,v)
print("O valor do valor-p para o t observado é de : {0}".format(valorp))


O número de graus de liberdade é de : 17
O valor de t observado é de : -0.2696883638058703
O valor do valor-p para o t observado é de : 0.395323146076426


Da tabela t student temos que para 95% de confiança com v graus de liberdade, tcritico = 1.740. Como estamos testando para 
valores maiores que u0, tcritico esta na cauda da direita. Já que tobservado < tcritico, não há evidências para refutar a H0. Outro jeito de comprovar isso é que o valor-p para tobservado é menor que 95%.

In [73]:
x = mediaA
u = mediaAB
s = desvioA
n = len(tempos_A)
#calculando graus de liberdade
v = n - 1
print("O número de graus de liberdade é de : {0}".format(v))
#testando para ver se versao A rejeita a hipotese
t = (x - u) / (s / (n ** (1 / 2)))
print("O valor de t observado é de : {0}".format(t))
#calculando o valor-p
valorp = scipy.stats.t.cdf(t,v)
print("O valor do valor-p para o t observado é de : {0}".format(valorp))

O número de graus de liberdade é de : 20
O valor de t observado é de : 0.12952338602410532
O valor do valor-p para o t observado é de : 0.5508813493785847


Da tabela t student temos que para 95% de confiança com v graus de liberdade, tcritico = 1.725. Como estamos testando para 
valores maiores que u0, tcritico esta na cauda da direita. Já que tobservado < tcritico, não há evidências para refutar a H0. Outro jeito de comprovar isso é que o valor-p para tobservado é menor que 95%.

# Conclusão

Baseado nos testes feitos e nos dados obtidos não podemos afirmar se a versão A do site é melhor ou pior que a versão B. Tivemos pequenas diferenças nas médias amostrais, que indicam que talvez o site B seria mais intuitivo, o usuário que utilizou  aversão B encontrou o projeto me média 4 segundos antes do usuário da versão A. Entretando como os testes de hipóteses de ambas as versões nao refutaram nosso H0 não podemos fazer inferências com relação a uma versão ser melhor que a outra. Para uma próxima etapa poderíamos focar melhor em como filtrar e corrigir erros no registro do tempo e obter um espaço amostral ainda maior para podermos ter mais segurança sobre o nosso teste